In [ ]:
import tensorflow as tf
import numpy as np
import cv2
from IPython.display import display, Image
from PIL import Image as PILImage
import io

# Load the pre-trained model
model = tf.saved_model.load('D:\\faster_rcnn_resnet50_v1_640x640_coco17_tpu-8\\saved_model')
infer = model.signatures['serving_default']

# Label map
label_map = {1: 'person', 3: 'car', 18: 'dog'}

# Load and preprocess image
image = cv2.imread("image.jpg")
assert image is not None, "Image not found. Please check the filename and path."
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
input_tensor = tf.convert_to_tensor(image_rgb, dtype=tf.uint8)
input_tensor = tf.expand_dims(input_tensor, axis=0)

# Run detection
detections = infer(input_tensor)
boxes = detections['detection_boxes'].numpy()[0]
scores = detections['detection_scores'].numpy()[0]
classes = detections['detection_classes'].numpy()[0].astype(int)

# Filter detections
threshold = 0.5
indices = np.where(scores >= threshold)[0]
filtered_boxes = boxes[indices]
filtered_classes = classes[indices]

# Draw boxes
image_np = image_rgb.copy()
h, w, _ = image_np.shape
for box, cls in zip(filtered_boxes, filtered_classes):
    ymin, xmin, ymax, xmax = box
    ymin, xmin, ymax, xmax = int(ymin * h), int(xmin * w), int(ymax * h), int(xmax * w)
    label = label_map.get(cls, 'Unknown')
    cv2.rectangle(image_np, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    cv2.putText(image_np, label, (xmin, max(0, ymin - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Convert to displayable format using PIL
output_pil = PILImage.fromarray(image_np)
buffer = io.BytesIO()
output_pil.save(buffer, format="PNG")
display(Image(data=buffer.getvalue()))
